In [1]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import namedtuple, deque
import random
import math
import numpy as np
from itertools import count
import matplotlib

from model import DQN


import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [3]:
"""
    this keeps track of preiovus moves for training
"""

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 32 # maybe try higher later
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

In [5]:
policy_net = DQN().to(device) # F(State, Action) -> Reward
target_net = DQN().to(device) # Maximize Reward
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(1000000)

In [6]:
steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            output = policy_net(state)
            # print(output, torch.argmax(output))
            return torch.argmax(output)
    else:
        return torch.tensor([np.random.choice(4)], dtype=torch.int64, device=device)

In [7]:
episode_durations = []

def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [8]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action).unsqueeze(1)
    reward_batch = torch.cat(batch.reward)
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    out = policy_net(state_batch)
    state_action_values = out.gather(1, action_batch)
    # print(out, action_batch, state_action_values)
    

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device, dtype=torch.int64)
    with torch.no_grad():
        next_state_values[non_final_mask] = torch.argmax(target_net(non_final_next_states))
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values)

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [9]:
import numpy as np
def get_reward(prev, next, successful):
    # if nothing occured then bad score
    if not successful:
        return -10000000
    
    # difference in average values times 5
    prev_vals = 0.0
    next_vals = 0.0
    total_prev = 0.0
    total_next = 0.0
    for i in range(prev.shape[0]):
        for j in range(prev.shape[1]):
            prev_vals += (prev[i, j] != 0)
            next_vals += (next[i, j] != 0)
            total_prev += prev[i, j]
            total_next += next[i, j]

    return 5 if (total_next / next_vals) > (total_prev / prev_vals) else -5
    

In [10]:
if torch.cuda.is_available() or torch.backends.mps.is_available():
    num_episodes = 2000
else:
    num_episodes = 50

import sys
sys.path.append("..") 
from Game_Sim import game

for i_episode in range(num_episodes):
    # Initialize the environment and get its state

    g = game.Game()
    state = g.board
    state = torch.tensor(state.flatten(), dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        prev = g.board
        action = select_action(state)
        successful = g.move(action)
        print("move is", action, " and is", successful)
        action = torch.tensor([action], dtype=torch.int64, device=device)
        observation = g.board
        reward = get_reward(prev, g.board, successful)
        terminated = g.check_win()
        truncated = g.check_loss()
        
        reward = torch.tensor([reward], dtype=torch.float32, device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
            print(prev)
        else:
            next_state = torch.tensor(observation.flatten(), dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

move is tensor(2, device='cuda:0')  and is True
move is tensor(2, device='cuda:0')  and is True
move is tensor([0], device='cuda:0')  and is True
move is tensor([0], device='cuda:0')  and is False
move is tensor([0], device='cuda:0')  and is False
move is tensor(2, device='cuda:0')  and is True
move is tensor([1], device='cuda:0')  and is True
move is tensor([1], device='cuda:0')  and is False
move is tensor([2], device='cuda:0')  and is True
move is tensor([1], device='cuda:0')  and is True
move is tensor([1], device='cuda:0')  and is False
move is tensor(2, device='cuda:0')  and is True
move is tensor(2, device='cuda:0')  and is True
move is tensor(2, device='cuda:0')  and is True
move is tensor([0], device='cuda:0')  and is True
move is tensor([1], device='cuda:0')  and is True
move is tensor([3], device='cuda:0')  and is True
move is tensor(2, device='cuda:0')  and is True
move is tensor([1], device='cuda:0')  and is True
move is tensor([2], device='cuda:0')  and is True
move is te

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [ ]:
target_net_state_dict = target_net.state_dict()
policy_net_state_dict = policy_net.state_dict()

torch.save(target_net_state_dict, 'prev_models/target_net.pt')
torch.save(policy_net_state_dict, 'prev_models/policy_net.pt')